# 복합 토픽 모델링(Combined Topic Modeling)

이 튜토리얼에서는 복합 토픽 모델(**Combined Topic Model**)을 사용하여 문서의 집합에서 토픽을 추출해보겠습니다.

## 토픽 모델(Topic Models)

토픽 모델을 사용하면 비지도 학습 방식으로 문서에 잠재된 토픽을 추출할 수 있습니다.

## 문맥을 반영한 토픽 모델(Contextualized Topic Models)
문맥을 반영한 토픽 모델(Contextualized Topic Models, CTM)이란 무엇일까요? CTM은 BERT 임베딩의 표현력과 토픽 모델의 비지도 학습의 능력을 결합하여 문서에서 주제를 가져오는 토픽 모델의 일종입니다.

# Contextualized Topic Models, CTM 설치

contextualized topic model 라이브러리를 설치합시다.

In [1]:
!pip install contextualized-topic-models==2.2.0

In [2]:
!pip install pyldavis

In [29]:
!pip install eunjeon

In [34]:
from eunjeon import Mecab
mecab = Mecab()

mecab.pos("품사 태깅을 지원합니다.")
mecab.morphs("형태소 분리를 지원합니다.")
mecab.nouns("명사에 해당하는 형태소만 추출합니다.")

['명사', '해당', '형태소', '추출']

## 노트북 재시작

원활한 실습을 위해서 노트북을 재시작 할 필요가 있습니다.

상단에서 런타임 > 런타임 재시작을 클릭해주세요.

# 데이터

학습을 위한 데이터가 필요합니다. 여기서는 하나의 라인(line)에 하나의 문서로 구성된 파일이 필요한데요. 우선, 여러분들의 데이터가 없다면 여기서 준비한 파일로 실습을 해봅시다.

In [4]:
!python -m wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt


Saved under 2016-10-20.txt


In [16]:
text_file = "2016-10-20.txt"

# 필요한 것들을 임포트

In [17]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

## 전처리

In [18]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [19]:
documents[:5]

['',
 '19  1990  52 1 22',
 '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽

In [20]:
not '19  1990  52 1 22'.replace(' ', '').isdecimal()

False

In [21]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|████████████████████████████████████████████████████████████████████████| 30091/30091 [00:00<00:00, 256548.96it/s]


In [22]:
len(preprocessed_documents)

27540

In [23]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [35]:
custom_tokenizer = CustomTokenizer(Mecab())

In [36]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [37]:
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

In [38]:
print(train_bow_embeddings.shape)

(27540, 3000)


In [39]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [40]:
len(vocab)

3000

In [41]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

In [44]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

## Combined TM 학습하기
이제 토픽 모델을 학습합니다. 여기서는 하이퍼파라미터에 해당하는 토픽의 개수(n_components)로는 50개를 선정합니다.

In [43]:
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [550800/550800]	Train Loss: 853.4902113261393	Time: 0:00:23.716638: : 20it [07:54, 23.74s/it]


# 토픽들

학습 후에는 토픽 모델이 선정한 토픽들을 보려면 아래의 메소드를 사용합니다.

```
get_topic_lists
```
해당 메소드에는 각 토픽마다 몇 개의 단어를 보고 싶은지에 해당하는 파라미터를 넣어즐 수 있습니다.

In [45]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['지연', '항공기', '정부', '국토', '항공'],
             1: ['공시', '왕자', '코어', '투데이', '머니'],
             2: ['연합뉴스', '우리나라', '사원', '강원도', '도와'],
             3: ['국회', '기권', '결의안', '찬성', '국감'],
             4: ['데일리', '진행', '기사', '패션쇼', '디자이너'],
             5: ['그룹', '멤버', '불독', '데뷔', '101'],
             6: ['청소년', '축제', '문화', '행사', '진로'],
             7: ['연합뉴스', '부여', '강원도', '형식', '통해서'],
             8: ['사랑', '제작', '때문', '배우', '차태현'],
             9: ['에게', '사람', '커플', '강호동', '이경규'],
             10: ['기온', '취업자', '매물', '원대', '10'],
             11: ['판결', '소송', '선고', '이혼', '지법'],
             12: ['84', '전용', '시흥', '시티', '지상'],
             13: ['이탈', '포토', '시리즈', '프리즘', '일간스포츠'],
             14: ['패션', '디자인', '서울', '2017', '위크'],
             15: ['코리아', '지진', '선고', '구미', '지방'],
             16: ['경찰', '터널', '경위', '살인', '김창호'],
             17: ['에서', '개구리', '한다', '아프리카', '으로'],
             18: ['습니다', '사람', '지만', '입니다', '발찌'],
             19: ['정유라',

In [46]:
ctm.get_topics(10)

defaultdict(list,
            {0: ['지연', '항공기', '정부', '국토', '항공', '노선', '운항', '레저', '선박', '항공사'],
             1: ['공시',
              '왕자',
              '코어',
              '투데이',
              '머니',
              '칼리드',
              '여의도',
              '코리아',
              '배정',
              '회견'],
             2: ['연합뉴스',
              '우리나라',
              '사원',
              '강원도',
              '도와',
              '최대한',
              '균형',
              '부여',
              '춘천',
              '한다면'],
             3: ['국회',
              '기권',
              '결의안',
              '찬성',
              '국감',
              '누리',
              '의원',
              '여야',
              '회고록',
              '송민순'],
             4: ['데일리',
              '진행',
              '기사',
              '패션쇼',
              '디자이너',
              '마이',
              '브랜드',
              '미디어',
              '디자인',
              '패션'],
             5: ['그룹', '멤버', '불독', '데뷔', '101', '프로', '스타', '뮤직',

# 시각화

우리의 토픽들을 시각화하기 위해서는 PyLDAvis를 사용합니다.

위에서 출력한 토픽 번호는 pyLDAvis에서 할당한 토픽 번호와 일치하지 않으므로 주의합시다.  
가령, 48번 토픽이었던 ['원유', '유가', '뉴욕', '오른', '연방', '마쳤', '서부', '달러', '51', '지수']가 아래의 PyLDAvis에서는 24번 토픽이 되었습니다.


In [47]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [03:03, 18.30s/it]
C:\Users\jikim\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


참고 자료 : https://github.com/MilaNLProc/contextualized-topic-models